# Prueba DE 

In [35]:
#Importar librerias generales prueba
import requests
import json
import numpy as np
import time
import os
import pandas as pd
from google.cloud import storage
import os
from datetime import datetime, timedelta,date
from google.cloud import bigquery
from dateutil.relativedelta import relativedelta
import random
import mysql.connector as mysql
from sqlalchemy import create_engine
import pymysql
import zipfile
import gcsfs

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [37]:
#credenciales cuenta de servicio
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\sjara11\Desktop\Santiago Jaramillo\Pruebas\Gluky\_credentials\sa_google_storage.json"

In [38]:
#inicializar el cliente GCS
storage_client = storage.Client()


In [39]:
# inicializar el cliente de BQ
client = bigquery.Client()


In [40]:
#Variables GCP
dataset_id = 'gluky_dataset'
project = 'daring-agent-324700'

# Punto 1. Usuarios

## API Twitter

In [ ]:
#Credenciales de acceso
def auth():
    with open(r"C:\Users\sjara11\Desktop\Santiago Jaramillo\Pruebas\Gluky\_credentials\tw_bearer_token_app.json", 'r') as file:
      creds = json.load(file)
      return creds["BEARER TOKEN"]

#Definir la variable de entorno
os.environ['BEARER_TOKEN'] = auth()

In [ ]:
#Parametro dia anterior
#UTC - 5 Colombia
yesterday = datetime.now() - timedelta(days=1)

#Ajuste horario para quedar en formato UTC
yesterday_i = (yesterday.replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(hours = 5)).isoformat() + 'Z'
yesterday_f = (yesterday.replace(hour=23, minute=59, second=59, microsecond=0)- timedelta(hours = 5)).isoformat() + 'Z'

day = yesterday.strftime('%Y-%m-%d')

print(yesterday_i,'\n',yesterday_f,'\n',day)


In [ ]:
#tomar la EV
bearer_token = os.environ.get('BEARER_TOKEN')

#url endpoint
search_url = "https://api.twitter.com/2/tweets/search/recent"

#Definir el query de los tweets
query_params = {'query':'MotoGP lang:es -is:retweet',
                'max_results':100,
                'user.fields':'id,location,name,username,verified,created_at',
                'end_time':yesterday_f,
                'start_time':yesterday_i,
                'expansions':'author_id',
                'tweet.fields': 'author_id'}



In [ ]:
#Funciones para autenticarse, contectarse y realizar la solicitud a la API
def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


In [ ]:
#Extraer el json y almacenarlo en una variable para cargar al area raw
api_response = connect_to_endpoint(search_url, query_params)

In [ ]:
#Extracción de la información de usuarios unicamente
users = api_response['includes']['users']


In [ ]:
#Pasar Json a Dataframe para ordenar los datos a guardar
df_users = pd.DataFrame(columns=['id','username','premiun','ubicacion','fecha_creacion', 'fecha_carga'])

i=0

for user in users:
    df_users.loc[i,'id'] = user['id']
    df_users.loc[i,'username'] = user['username']
    df_users.loc[i,'premiun'] = user['verified']
    df_users.loc[i,'fecha_creacion'] = user['created_at'].split('T')[0]
    df_users.loc[i,'fecha_carga'] = day
    

    if 'location' in user.keys():
        df_users.loc[i,'ubicacion'] = user['location']
    else:
        df_users.loc[i,'ubicacion'] = 'sin_location'
        
    i += 1

In [ ]:
#Validación de cantidad de usuarios tomados
len(df_users)

In [ ]:
#nombre archivo
filename = f'users_{day}'

In [ ]:
#Guardar el csv
df_users.to_csv(r"C:\Users\sjara11\Desktop\Santiago Jaramillo\Pruebas\Gluky\output\{}.csv".format(filename), index= False, compression = 'gzip')

## Cargar archivo a GoogleStorage 

In [ ]:
#Listar los buckets
buckets = list(storage_client.list_buckets())
buckets

In [ ]:
#Carga de archivo al bucket
bucket = storage_client.get_bucket("prueba_gluky")

blob = bucket.blob('raw_zone/users_{}'.format(day))
blob.upload_from_filename(r'C:\Users\sjara11\Desktop\Santiago Jaramillo\Pruebas\Gluky\output\{}.csv'.format(filename))

## Creación dimension de usuarios en BigQuery

### Crear la tabla externa permanente

### Crear la vista con los usuarios únicos 

# Punto 2. Reporte ventas

# Punto 3. Usuarios & Puntos

### Funciones para la conexión a MySQL

In [21]:
db = 'gluky_test'

#Creación del engine
def mysql_engine(user = 'root', password = '', host = '127.0.0.1', port = '3306', database = db):
    engine = create_engine("mysql+pymysql://{0}:{1}@{2}:{3}/{4}?charset=utf8".format(user, password, host, port, database))
    return engine


#Creación de la conexión
def mysql_conn():
    conn = mysql.connect(host="localhost",database = db, user="root",password="")
    return conn


#Crear y escribir la tabla remplazando la anterior
def df_to_mysql(df, engine, tbl_name):
    df.to_sql(tbl_name, engine, if_exists='replace')


## Simulación de puntos

In [ ]:
#Tomar los usuarios que estan almacenados en BQ.
query_string =  """
    SELECT distinct(username)
    FROM `daring-agent-324700.gluky_dataset.raw_usuarios`
"""

usernames = (
    client.query(query_string)
    .result()
    .to_dataframe(create_bqstorage_client=True,)
)
print(usernames.head())


In [ ]:
#Simulacion aleatoria del dataframe con los puntos
conceptos = ['liquidacion','redencion']
usuarios = usernames['username'].to_list()
fecha_i  = date(2020, 1, 1)
random.seed(123)

puntos = pd.DataFrame(columns=['username','concepto','valor','fecha'])
asignaciones = pd.DataFrame(columns=['username','concepto','valor','fecha'])
retiros = pd.DataFrame(columns=['username','concepto','valor','fecha'])

i = 0
#Creación de 500 asignaciones
for x in range(0,500):
    asignaciones.loc[i,'username'] = usuarios[random.randint(0,len(usuarios)-1)]
    asignaciones.loc[i,'concepto'] = conceptos[0]
    asignaciones.loc[i,'valor'] = random.randint(15000,50000)
    asignaciones.loc[i,'fecha'] = fecha_i + relativedelta(months= random.randint(0,12))
    
    i+=1

control_neg = []
#Creacion de 200 redenciones
i= 0
y = 0
while y < 200:
    fila = random.randint(0,len(asignaciones)-1)
    factor = round(random.random(),2)
    if fila in control_neg:
        pass
    else:
        retiros.loc[i,'username'] = asignaciones.loc[fila,'username']
        retiros.loc[i,'concepto'] = conceptos[1]
        retiros.loc[i,'valor'] = asignaciones.loc[fila,'valor']*factor*-1
        retiros.loc[i,'fecha'] = asignaciones.loc[fila,'fecha'] + relativedelta(months=random.randint(1,3))
        y+=1
    i+=1
    control_neg.append(fila)

In [ ]:
#Agregar las simulaciones, ordenar por fecha y guardar historico para subir a MySQL
puntos = pd.concat([asignaciones, retiros])
puntos.sort_values('fecha', inplace =True)
#puntos.to_excel(r"C:\Users\sjara11\Desktop\Santiago Jaramillo\Pruebas\Gluky\puntos\simulacion_puntos.xlsx")

In [ ]:
#Corrección de tipos de datos para la creación de la tabla en MySQL
puntos['username'] = puntos['username'].astype('string')
puntos['concepto'] = puntos['concepto'].astype('string')
puntos['valor'] = puntos['valor'].astype('float')
puntos['fecha'] = pd.to_datetime(puntos['fecha'],format= '%Y-%m-%d')

In [ ]:
#Definicion nombre de la tabla
db_tbl_name = 'puntos_sim'

#cargar tabla
df_to_mysql(puntos, mysql_engine(), db_tbl_name)

## Proceso de consulta de fuentes para unificar

### Consultar los datos de MYSQL

In [ ]:
#Realizar la consulta personalizada para traer los puntos resumidos por mes y usuario
conn = mysql_conn()
cursor = conn.cursor()

sql= """
SELECT username, fecha, PuntosRecibidos, PuntosRedimidos, SUM(PuntosRecibidos + PuntosRedimidos)  OVER(PARTITION BY username ORDER BY fecha)AS Saldo
FROM(
SELECT username,
fecha,
 SUM(CASE
        WHEN valor > 0 THEN valor
        ELSE 0
        END) AS PuntosRecibidos,
 SUM(CASE
        WHEN valor < 0 THEN valor
        ELSE 0
        END) AS PuntosRedimidos
FROM gluky_test.puntos
GROUP BY username, fecha
ORDER BY username, fecha
) as t
GROUP BY username, fecha
ORDER BY  username, fecha;

"""
cursor.execute(sql)
cursor.fetchall()

hist_puntos = pd.read_sql(sql,conn)

#Validar los datos cargados
hist_puntos.info()


In [ ]:
#Cerrar la conexion con mysql
conn.close()
cursor.close()

### Consultar los datos de los usuarios BQ

In [ ]:
#Tomar los usuarios que estan almacenados en BQ.
query_string =  """
    SELECT *
    FROM `daring-agent-324700.gluky_dataset.dim_usuarios`
"""

usuarios = (
    client.query(query_string)
    .result()
    .to_dataframe(create_bqstorage_client=True,)
)
usuarios.info()

### Resumir, unir y cargar la información

In [ ]:
#Traer al resumen de puntos la información cualitativa de los usuarios
hist_puntos_report = hist_puntos.merge(usuarios,how = 'left', on = 'username', validate='m:1')


In [ ]:
#Guardar el csv sin comprimir pra que data studio lo pueda leer del bucket
hist_puntos_report.to_csv(r'C:\Users\sjara11\Desktop\Santiago Jaramillo\Pruebas\Gluky\output\puntos_report.csv', index= False)

In [ ]:
#Carga de archivo al bucket
bucket = storage_client.get_bucket("prueba_gluky")

blob = bucket.blob('report/reporte_puntos')
blob.upload_from_filename(r'C:\Users\sjara11\Desktop\Santiago Jaramillo\Pruebas\Gluky\output\puntos_report.csv')

In [ ]:
Link de acceso al reporte de puntos
https://datastudio.google.com/s/rSnkQReV8HI

# Punto 4. Regla de negocio para calculo de puntos

In [3]:
#Para este ejemplo tome 10.000 datos del dataset publico de venta de licor usado en el punto 2, regitros todos mayores al 2019
#Carga de los datos de ejemplo para diseñar la funcion que tome la regla de negocio y calcule los puntos
sales = pd.read_csv(r"C:\Users\sjara11\Desktop\Santiago Jaramillo\Pruebas\Gluky\results-20210905-145605.csv")

In [7]:
#Funcion para crear las metas por producto, default por botellas pero se puede cambiar a litros, dolares, etc
def creador_metas(variable = 'bottles_sold'):
    sales_agg = sales.loc[:,['vendor_name','date','item_description','bottles_sold','sale_dollars','volume_sold_liters']].groupby(['vendor_name','item_description','date'], as_index = False).mean()
    metas = sales_agg.groupby(['item_description']).mean()[variable].to_dict()
    return metas

metas = creador_metas()

In [8]:


#funcion para aplicar regla de negocio variable por fecha
def cumplimiento(item, valor, puntos_a_otorgar, metas = metas):
    if valor >= metas[item]:
        puntos = puntos_a_otorgar
    else:
        puntos = 0
        
    return puntos
    
meses_all = list(range(1,13))
annos_all = list(range(1990,int(datetime.now().year)+1))

#funcion para asignar los puntos por las ventas
def liquidacion(data, puntos,meses = meses_all, annos = annos_all):
    sales = data
    #segmentar la info al periodo solicitado
    sales['date'] = pd.to_datetime(sales['date'], format = '%Y-%m-%d')
    sales['month'] = sales['date'].apply(lambda x: x.month)
    sales['year'] = sales['date'].apply(lambda x: x.year)
    sales = sales.loc[sales['month'].isin(meses) & sales['year'].isin(annos),:]
    
    #agregar la información para totalizar las ventas por item, vendor y mes
    sales_agg = sales.loc[:,['vendor_name','date','item_description','category_name','bottles_sold','sale_dollars']].groupby(['vendor_name','item_description','category_name','date'], as_index = False).sum()
    sales_agg['puntos'] = sales_agg.apply(lambda x: cumplimiento(x['item_description'],x['bottles_sold'],puntos), axis = 1)

    return sales_agg


In [9]:
#Ejecución de la liquidación para enero, febrero y marzo del 2020 entregando 230 puntos por cumplir la ventas mensuales por producto
liquidacion(sales,230,[1,2,3],[2020])

,vendor_name,item_description,category_name,date,bottles_sold,sale_dollars,puntos
0,3 BADGE CORPORATION,SOOH Uncle Vals Botanical Gin,Flavored Gin,2020-03-02,1,28.88,0
1,"American Heritage Distillers, LLC",Century Farms Prairie States Corn Whiskey Rapi...,American Distilled Spirit Specialty,2020-02-18,4,51.08,0
2,"American Heritage Distillers, LLC",Century Farms Prairie States Corn Whiskey Rapi...,American Distilled Spirit Specialty,2020-03-18,4,51.08,0
3,BACARDI USA INC,Bacardi 8,Aged Dark Rum,2020-03-12,1,22.50,0
4,BACARDI USA INC,Bacardi Anejo Cuatro,Gold Rum,2020-02-13,1,14.99,0
5,BACARDI USA INC,Bacardi Anejo Cuatro,Gold Rum,2020-03-05,1,14.99,0
6,BACARDI USA INC,Bacardi Black,Aged Dark Rum,2020-02-13,6,135.00,0
7,BACARDI USA INC,Bacardi Black,Aged Dark Rum,2020-03-24,1,22.50,0
8,BACARDI USA INC,Bacardi Black,Aged Dark Rum,2020-03-31,12,270.00,230
9,BACARDI USA INC,Bacardi Gold,Gold Rum,2020-02-04,3,40.89,0


# Punto 5. Pipeline de datos & Analitica

## Carga masiva a GCS 

## Extracción a MySQL Staging


In [43]:
source_bucket = storage_client.bucket('prueba_gluky')

col_names = ['ts',
 'eventName',
 'profile.identity',
 'profile.objectId',
 'profile.all_identities',
 'profile.platform',
 'profile.phone',
 'profile.name',
 'profile.email',
 'profile.push_token',
 'deviceInfo.make',
 'deviceInfo.model',
 'deviceInfo.appVersion',
 'deviceInfo.sdkVersion',
 'deviceInfo.osVersion',
 'deviceInfo.browser',
 'deviceInfo.dpi',
 'deviceInfo.dimensions.width',
 'deviceInfo.dimensions.height',
 'deviceInfo.dimensions.unit',
 'controlGroupName',
 'eventProps.CT Source',
 'eventProps.wzrk_cts',
 'eventProps.wzrk_push_amp',
 'eventProps.wzrk_ck',
 'eventProps.wzrk_from',
 'eventProps.UTM',
 'eventProps.wzrk_bi_liquid',
 'eventProps.wzrk_c2a',
 'eventProps.wzrk_pid',
 'eventProps.Campaign id',
 'eventProps.Install',
 'eventProps.CT App Version',
 'eventProps.Campaign type',
 'eventProps.wzrk_dl_liquid',
 'eventProps.wzrk_acct_id',
 'eventProps.wzrk_cid',
 'eventProps.wzrk_dl',
 'eventProps.wzrk_pivot',
 'eventProps.wzrk_bi',
 'eventProps.wzrk_bc',
 'eventProps.wzrk_rnv',
 'eventProps.wzrk_bc_liquid',
 'eventProps.RAW_STRING',
 'eventProps.wzrk_pn',
 'eventProps.wzrk_cid_liquid',
 'eventProps.wzrk_id',
 'eventProps.wzrk_nms_liquid',
 'eventProps.wzrk_ttl',
 'eventProps.wzrk_dt']

df_consolidado = pd.DataFrame(columns=col_names)

for file in list(source_bucket.list_blobs()):
    if file.name.split('/')[0] == 'pipeline':
        file_path="gs://{}/{}".format(file.bucket.name, file.name)
        df_consolidado = df_consolidado.append(pd.read_csv(file_path, header = 1 , names=col_names))
    else:
        pass

In [33]:
#Validación tipos de datos
df_consolidado.dtypes

ts                               object
eventName                        object
profile.identity                 object
profile.objectId                float64
profile.all_identities           object
profile.platform                 object
profile.phone                    object
profile.name                     object
profile.email                    object
profile.push_token               object
deviceInfo.make                  object
deviceInfo.model                 object
deviceInfo.appVersion            object
deviceInfo.sdkVersion            object
deviceInfo.osVersion             object
deviceInfo.browser               object
deviceInfo.dpi                   object
deviceInfo.dimensions.width      object
deviceInfo.dimensions.height     object
deviceInfo.dimensions.unit       object
controlGroupName                float64
eventProps.CT Source             object
eventProps.wzrk_cts             float64
eventProps.wzrk_push_amp         object
eventProps.wzrk_ck              float64


In [44]:
#Creacion de indice único
df_consolidado.reset_index(inplace = True, drop = True)
df_consolidado.head()

,ts,eventName,profile.identity,profile.objectId,profile.all_identities,profile.platform,profile.phone,profile.name,profile.email,profile.push_token,deviceInfo.make,deviceInfo.model,deviceInfo.appVersion,deviceInfo.sdkVersion,deviceInfo.osVersion,deviceInfo.browser,deviceInfo.dpi,deviceInfo.dimensions.width,deviceInfo.dimensions.height,deviceInfo.dimensions.unit,controlGroupName,eventProps.CT Source,eventProps.wzrk_cts,eventProps.wzrk_push_amp,eventProps.wzrk_ck,eventProps.wzrk_from,eventProps.UTM,eventProps.wzrk_bi_liquid,eventProps.wzrk_c2a,eventProps.wzrk_pid,eventProps.Campaign id,eventProps.Install,eventProps.CT App Version,eventProps.Campaign type,eventProps.wzrk_dl_liquid,eventProps.wzrk_acct_id,eventProps.wzrk_cid,eventProps.wzrk_dl,eventProps.wzrk_pivot,eventProps.wzrk_bi,eventProps.wzrk_bc,eventProps.wzrk_rnv,eventProps.wzrk_bc_liquid,eventProps.RAW_STRING,eventProps.wzrk_pn,eventProps.wzrk_cid_liquid,eventProps.wzrk_id,eventProps.wzrk_nms_liquid,eventProps.wzrk_ttl,eventProps.wzrk_dt
0,20201211132441,Notification Clicked,11710313,NaN,"[11710313, 11710313@nomail.gluky.com]",Android,573223722354,Jose Joaquin Ortiz Vargas,11710313@nomail.gluky.com,fcm:fTajFDCX99Q:APA91bGY86L8qEKY2FcLBrQVlb-DVD...,huawei,JKM-LX3,3.3.0,30601,9,MobileApp,480,69,136,mm,NaN,Mobile,NaN,False,NaN,NaN,NaN,NaN,NaN,20138_20201211,NaN,NaN,3.3.0,Push,NaN,WWR-Z94-775Z,Elegidos,elegidosbavaria://elegidos.com/app/beneficios/...,wzrk_default,2,NaN,False,NaN,NaN,True,NaN,20138_20201211,NaN,1607797412,FIREBASE
1,20201211133057,Notification Clicked,12118896,NaN,"[12118896, yanpool83@gmail.com]",Android,573107605196,12118896,yanpool83@gmail.com,fcm:cf6Ko5qHDi0:APA91bH2D8AukujzE8nHHajGHCUZLR...,SAMSUNG,SM-A115M,3.3.0,30601,10,MobileApp,280,56,112,mm,NaN,Mobile,NaN,False,NaN,NaN,NaN,NaN,NaN,20138_20201211,NaN,NaN,3.3.0,Push,NaN,WWR-Z94-775Z,Elegidos,elegidosbavaria://elegidos.com/app/beneficios/...,wzrk_default,2,NaN,False,NaN,NaN,True,NaN,20138_20201211,NaN,1607797412,FIREBASE
2,20201211134729,Notification Clicked,11507474,NaN,"[11507474, maryfonsecalar08@gmail.com]",Android,573123417241,11507474,maryfonsecalar08@gmail.com,fcm:cwt0jwbHaNc:APA91bFESQNFCVmIasuJmFNm-oXKne...,SAMSUNG,SM-J610G,3.3.0,30601,10,MobileApp,280,66,129,mm,NaN,Mobile,NaN,False,NaN,NaN,NaN,NaN,NaN,20138_20201211,NaN,NaN,3.3.0,Push,NaN,WWR-Z94-775Z,Elegidos,elegidosbavaria://elegidos.com/app/beneficios/...,wzrk_default,2,NaN,False,NaN,NaN,True,NaN,20138_20201211,NaN,1607797412,FIREBASE
3,20201211154152,Notification Clicked,11710313,NaN,"[11710313, 11710313@nomail.gluky.com]",Android,573223722354,Jose Joaquin Ortiz Vargas,11710313@nomail.gluky.com,fcm:fTajFDCX99Q:APA91bGY86L8qEKY2FcLBrQVlb-DVD...,huawei,JKM-LX3,3.3.0,30601,9,MobileApp,480,69,136,mm,NaN,Mobile,NaN,False,NaN,NaN,NaN,NaN,NaN,20142_20201211,NaN,NaN,3.3.0,Push,NaN,WWR-Z94-775Z,Elegidos,elegidosbavaria://elegidos.com/app/beneficios/...,wzrk_default,2,NaN,False,NaN,NaN,True,NaN,20142_20201211,NaN,1607805370,FIREBASE
4,20201211154348,Notification Clicked,11507474,NaN,"[11507474, maryfonsecalar08@gmail.com]",Android,573123417241,11507474,maryfonsecalar08@gmail.com,fcm:cwt0jwbHaNc:APA91bFESQNFCVmIasuJmFNm-oXKne...,SAMSUNG,SM-J610G,3.3.0,30601,10,MobileApp,280,66,129,mm,NaN,Mobile,NaN,False,NaN,NaN,NaN,NaN,NaN,20142_20201211,NaN,NaN,3.3.0,Push,NaN,WWR-Z94-775Z,Elegidos,elegidosbavaria://elegidos.com/app/beneficios/...,wzrk_default,2,NaN,False,NaN,NaN,True,NaN,20142_20201211,NaN,1607805370,FIREBASE


In [45]:
#Definicion nombre de la tabla
db_tbl_name = 'stg_mkt_info'

#cargar tabla
df_to_mysql(df_consolidado, mysql_engine(), db_tbl_name)